# 🌦️ Module 1: การวิเคราะห์ข้อมูลอุตุนิยมวิทยาด้วย Google Earth Engine
## Meteorological Data Analysis with GEE Python API

**Workshop: การใช้ GEE วิเคราะห์ข้อมูลอุตุ-อุทก-ไฟป่า-คุณภาพน้ำ**

---

### 📋 เนื้อหาในบทนี้
1. เชื่อมต่อ Google Earth Engine
2. ข้อมูลปริมาณฝน (CHIRPS)
3. ข้อมูลอุณหภูมิ (ERA5-Land)
4. ข้อมูลสภาพอากาศ (ERA5 Monthly)
5. การสร้างกราฟ Time Series
6. การทำแผนที่สภาพอากาศ

### 📌 พื้นที่ศึกษา: ประเทศไทย
### ⏱️ เวลา: ~45 นาที (9:00 - 9:45)

---
## 1. ติดตั้งและเชื่อมต่อ GEE

In [ ]:
# ติดตั้ง geemap (ถ้ายังไม่มี)
# !pip install geemap earthengine-api --quiet

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# ตั้งค่า font สำหรับภาษาไทย (ถ้ามี)
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Authenticate and Initialize
# วิธีที่ 1: สำหรับ Colab
ee.Authenticate()
ee.Initialize(project='YOUR-PROJECT-ID')  # <-- ใส่ GEE Cloud Project ID ของคุณ

print('GEE เชื่อมต่อสำเร็จ ✅')

---
## 2. กำหนดพื้นที่ศึกษา (Area of Interest)

In [ ]:
# ขอบเขตประเทศไทย จาก FAO GAUL
thailand = ee.FeatureCollection('FAO/GAUL/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

# ขอบเขตจังหวัด (ระดับ 1)
provinces = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

# ตัวอย่าง: เลือกจังหวัดเชียงใหม่
chiang_mai = provinces.filter(ee.Filter.eq('ADM1_NAME', 'Chiang Mai'))

# แสดงแผนที่
Map = geemap.Map(center=[15.87, 100.99], zoom=6)
Map.addLayer(thailand, {'color': 'blue'}, 'Thailand')
Map.addLayer(chiang_mai, {'color': 'red'}, 'Chiang Mai')
Map

---
## 3. ข้อมูลปริมาณฝน - CHIRPS Daily

**CHIRPS** (Climate Hazards Group InfraRed Precipitation with Station data)  
- ความละเอียด: 0.05° (~5.5 km)  
- ช่วงเวลา: 1981 - ปัจจุบัน  
- หน่วย: mm/day  
- Dataset: `UCSB-CHG/CHIRPS/DAILY`

In [ ]:
# โหลดข้อมูล CHIRPS รายวัน ปี 2024
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(thailand)

print(f'จำนวนภาพ CHIRPS: {chirps.size().getInfo()} ภาพ')

In [ ]:
# คำนวณปริมาณฝนรวมรายเดือน
months = ee.List.sequence(1, 12)

def monthly_rainfall(month):
    month = ee.Number(month)
    monthly = chirps.filter(
        ee.Filter.calendarRange(month, month, 'month')
    ).sum()
    return monthly.set('month', month).set('system:time_start',
        ee.Date.fromYMD(2024, month, 1).millis())

monthly_rain = ee.ImageCollection.fromImages(months.map(monthly_rainfall))
print('สร้างข้อมูลฝนรายเดือนสำเร็จ ✅')

In [ ]:
# แสดงแผนที่ฝนสะสมทั้งปี 2024
annual_rain = chirps.sum().clip(thailand)

vis_rain = {
    'min': 500,
    'max': 2500,
    'palette': ['white', 'lightblue', 'blue', 'darkblue', 'purple']
}

Map2 = geemap.Map(center=[15.87, 100.99], zoom=6)
Map2.addLayer(annual_rain, vis_rain, 'Annual Rainfall 2024 (mm)')
Map2.add_colorbar(vis_rain, label='Rainfall (mm)', layer_name='Annual Rainfall 2024 (mm)')
Map2

In [ ]:
# สร้างกราฟ Time Series ปริมาณฝนรายเดือน (เชียงใหม่)
def get_monthly_mean(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=chiang_mai.geometry(),
        scale=5000,
        maxPixels=1e9
    )
    return image.set('rain_mean', stats.get('precipitation'))

monthly_stats = monthly_rain.map(get_monthly_mean)

# ดึงข้อมูลเป็น list
rain_values = monthly_stats.aggregate_array('rain_mean').getInfo()
month_labels = ['ม.ค.', 'ก.พ.', 'มี.ค.', 'เม.ย.', 'พ.ค.', 'มิ.ย.',
                'ก.ค.', 'ส.ค.', 'ก.ย.', 'ต.ค.', 'พ.ย.', 'ธ.ค.']

# พล็อตกราฟ
fig, ax = plt.subplots(figsize=(12, 5))
bars = ax.bar(month_labels, rain_values, color='steelblue', edgecolor='navy', alpha=0.8)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Rainfall (mm)', fontsize=12)
ax.set_title('Monthly Rainfall - Chiang Mai 2024 (CHIRPS)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# ใส่ค่าบนแท่ง
for bar, val in zip(bars, rain_values):
    if val is not None:
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 5,
                f'{val:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

---
## 4. ข้อมูลอุณหภูมิ - ERA5-Land Monthly

**ERA5-Land** (ECMWF Reanalysis v5 - Land)  
- ความละเอียด: ~11 km  
- ช่วงเวลา: 1950 - ปัจจุบัน  
- Band: `temperature_2m` (Kelvin)  
- Dataset: `ECMWF/ERA5_LAND/MONTHLY_AGGR`

In [ ]:
# โหลดข้อมูล ERA5-Land Monthly
era5 = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(thailand) \
    .select('temperature_2m')

print(f'จำนวนภาพ ERA5: {era5.size().getInfo()} ภาพ')

In [ ]:
# แปลง Kelvin เป็น Celsius
def kelvin_to_celsius(image):
    return image.subtract(273.15) \
        .copyProperties(image, ['system:time_start'])

era5_celsius = era5.map(kelvin_to_celsius)

# แสดงอุณหภูมิเฉลี่ยเดือนเมษายน (เดือนร้อนที่สุด)
april_temp = era5_celsius.filter(
    ee.Filter.calendarRange(4, 4, 'month')
).first().clip(thailand)

vis_temp = {
    'min': 20,
    'max': 35,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}

Map3 = geemap.Map(center=[15.87, 100.99], zoom=6)
Map3.addLayer(april_temp, vis_temp, 'April 2024 Temperature (°C)')
Map3.add_colorbar(vis_temp, label='Temperature (°C)', layer_name='April 2024 Temperature (°C)')
Map3

In [ ]:
# กราฟ Time Series อุณหภูมิรายเดือน
def get_temp_mean(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=thailand.geometry(),
        scale=11000,
        maxPixels=1e9
    )
    return image.set('temp_mean', stats.get('temperature_2m'))

temp_stats = era5_celsius.map(get_temp_mean)
temp_values = temp_stats.aggregate_array('temp_mean').getInfo()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(month_labels, temp_values, 'ro-', linewidth=2, markersize=8, label='Mean Temp')
ax.fill_between(range(len(temp_values)), temp_values, alpha=0.2, color='red')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Temperature (°C)', fontsize=12)
ax.set_title('Monthly Mean Temperature - Thailand 2024 (ERA5-Land)', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
ax.legend()

for i, val in enumerate(temp_values):
    if val is not None:
        ax.annotate(f'{val:.1f}°C', (i, val), textcoords='offset points',
                    xytext=(0, 10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---
## 5. Anomaly Analysis: เปรียบเทียบฝนกับค่าเฉลี่ย

เปรียบเทียบปริมาณฝนปี 2024 กับค่าเฉลี่ย 30 ปี (Climate Normal)

In [ ]:
# คำนวณค่าเฉลี่ยฝนรายปี 30 ปี (1991-2020)
chirps_climatology = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate('1991-01-01', '2020-12-31') \
    .filterBounds(thailand)

# ฝนเฉลี่ยรายปี (30 ปี)
annual_mean_30yr = chirps_climatology.sum() \
    .divide(30) \
    .clip(thailand)

# ฝนปี 2024
annual_2024 = chirps.sum().clip(thailand)

# คำนวณ Anomaly (ผลต่าง)
rain_anomaly = annual_2024.subtract(annual_mean_30yr)

vis_anomaly = {
    'min': -500,
    'max': 500,
    'palette': ['brown', 'orange', 'white', 'lightblue', 'darkblue']
}

Map4 = geemap.Map(center=[15.87, 100.99], zoom=6)
Map4.addLayer(rain_anomaly, vis_anomaly, 'Rainfall Anomaly 2024 (mm)')
Map4.add_colorbar(vis_anomaly, label='Anomaly (mm)', layer_name='Rainfall Anomaly 2024 (mm)')
Map4

---
## 6. การเปรียบเทียบฝนระหว่างจังหวัด

In [ ]:
# เปรียบเทียบฝนรายเดือนระหว่างภาค
province_list = ['Chiang Mai', 'Bangkok Metropolis', 'Nakhon Ratchasima',
                 'Surat Thani', 'Udon Thani']

fig, ax = plt.subplots(figsize=(14, 6))

for prov_name in province_list:
    prov = provinces.filter(ee.Filter.eq('ADM1_NAME', prov_name))

    def get_prov_rain(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=prov.geometry(),
            scale=5000,
            maxPixels=1e9
        )
        return image.set('rain', stats.get('precipitation'))

    prov_rain = monthly_rain.map(get_prov_rain)
    values = prov_rain.aggregate_array('rain').getInfo()
    ax.plot(month_labels, values, 'o-', linewidth=2, markersize=6, label=prov_name)

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Rainfall (mm)', fontsize=12)
ax.set_title('Monthly Rainfall Comparison by Province - 2024', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 7. Export ข้อมูลเป็น GeoTIFF

In [ ]:
# Export ฝนรวมทั้งปีไปยัง Google Drive
task = ee.batch.Export.image.toDrive(
    image=annual_rain,
    description='Thailand_Annual_Rainfall_2024',
    folder='GEE_Workshop',
    fileNamePrefix='thailand_rainfall_2024',
    region=thailand.geometry(),
    scale=5000,
    crs='EPSG:4326',
    maxPixels=1e10
)
task.start()
print('เริ่ม Export แล้ว ✅ ตรวจสอบใน Google Drive > GEE_Workshop')

---
## 📝 สรุป Module 1

| หัวข้อ | Dataset | Band | หน่วย | Resolution |
|--------|---------|------|-------|------------|
| ปริมาณฝน | CHIRPS Daily | precipitation | mm/day | ~5.5 km |
| อุณหภูมิ | ERA5-Land Monthly | temperature_2m | Kelvin | ~11 km |

### 💡 Tips
- ใช้ `.filterDate()` เพื่อเลือกช่วงเวลา
- ใช้ `.filterBounds()` เพื่อเลือกพื้นที่
- ใช้ `.reduceRegion()` เพื่อสรุปค่าสถิติ
- ใช้ `.map()` เพื่อคำนวณซ้ำกับทุกภาพ

### 📚 แหล่งข้อมูลเพิ่มเติม
- [CHIRPS](https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY)
- [ERA5-Land](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY_AGGR)
- [geemap Documentation](https://geemap.org/)

---
**➡️ ต่อไป: Module 2 - อุทกวิทยา (Hydrology)**